# Create the C -> A dataset to be used by the fine-tuned BERT classifier

## Questions and thoughts
- Tutorial: https://huggingface.co/docs/transformers/custom_datasets
- Context texts must be limited to 512 tokens (Limit for BERT model)
- When labeling the dataset, should the labels be start, end, or start and inside? In other projects (with answer extraction) it seems they use start, end..
- Another option is to insert a higlight token around the sentence containing the answer, and then append the answers after a [SEP] token. As in: 
- There are multiple answer spans in the same context text.. Should those be labeled jointly? / should I have multiple instances of the same texts?
- My idea is to use the original text, no stopword removal or lemmatization.

In [62]:
# necessary library imports
import pandas as pd
import numpy as np
import math

In [64]:
# data imports, to be combined into the final datastructure
CA_df = pd.read_pickle("./data/CA/labeled_CA_data_train.pkl")
CAR_df = pd.read_pickle("./data/CAR/labeled_CAR_data_train.pkl")
CAR_sent_class_df = pd.read_pickle("./data/CAR_classification/labeled_CAR_data_train.pkl")
CRA_df = pd.read_pickle("./data/CRA/labeled_CRA_data_train.pkl")

In [76]:
# compute the class weights to use in the training of the C -> A model (to account for the scarse dataset)
# idea for how to scale weights:
# https://www.tensorflow.org/tutorials/structured_data/imbalanced_data#calculate_class_weights
# https://medium.com/gumgum-tech/handling-class-imbalance-by-introducing-sample-weighting-in-the-loss-function-3bdebd8203b4

def get_class_distribution(labeled_df, is_sent_class):
    if is_sent_class:
        nr_classes = 2
    else:
        nr_classes = 3
    counts = np.zeros(nr_classes)
    for idx, point in labeled_df.iterrows():
        if 'labels' in point.keys():
            labels = point['labels']
            for label in labels:
                counts[int(label)] += 1


        else:
            label = point['label']
            counts[int(label)] += 1

    num_labels = np.sum(counts)
    ins_weights = (num_labels/2) / counts
    ins_weights_raw = 1/ counts
    ins_weights_norm = ins_weights_raw / np.sum(ins_weights_raw) * nr_classes
    # ins_weights = np.array([1/num_zeros, 1/num_ones, 1/num_twos]) * (num_labels/2)
    # ins_weights_raw = np.array([1/num_zeros, 1/num_ones, 1/num_twos])
    # ins_weights_norm = ins_weights_raw / np.sum(ins_weights_raw) * nr_classes
    isns_weights_raw = 1 / np.sqrt(counts)
    isns_weights = isns_weights_raw * (math.sqrt(num_labels/2))
    # isns_weights = np.array([1/math.sqrt(num_zeros), 1/math.sqrt(num_ones), 1/math.sqrt(num_twos)]) * (math.sqrt(num_labels/2))
    # isns_weights_raw = np.array([1/math.sqrt(num_zeros), 1/math.sqrt(num_ones), 1/math.sqrt(num_twos)])
    isns_weights_norm = isns_weights_raw / np.sum(isns_weights_raw) * nr_classes

    # ENS
    B = 0.9999
    E_nc = (1.0 - np.power(B, counts)) / (1.0 - B)
    w = 1/E_nc
    # normalize:
    w = w / np.sum(w) * nr_classes

    # norm = np.linalg.norm(weights)
    # normal_array = weights/norm
    print('INS: ',ins_weights)
    print('INS, norm: ',ins_weights_norm)
    print('ISNS: ',isns_weights)
    print('ISNS, norm: ',isns_weights_norm)
    print('ENS: ',w)




In [77]:
get_class_distribution(CA_df, False)

INS:  [ 0.51416189 72.53486395 24.2126029 ]
INS, norm:  [0.01585914 2.23731182 0.74682904]
ISNS:  [0.71705083 8.51674022 4.92063034]
ISNS, norm:  [0.15197742 1.80510527 1.04291731]
ENS:  [0.22420819 2.02073775 0.75505406]


In [78]:
get_class_distribution(CAR_df, False)

INS:  [  0.54466864 110.06202837   6.45429684]
INS, norm:  [0.01395859 2.82063285 0.16540856]
ISNS:  [ 0.73801669 10.49104515  2.54053082]
ISNS, norm:  [0.16079271 2.28569837 0.55350893]
ENS:  [0.54060372 1.91688386 0.54251243]


In [79]:
get_class_distribution(CRA_df, False)

INS:  [  0.5034363  289.26768868  98.09344175]
INS, norm:  [0.00389391 2.23738681 0.75871928]
ISNS:  [ 0.70953245 17.00787137  9.90421333]
ISNS, norm:  [0.07706273 1.84723486 1.0757024 ]
ENS:  [0.23868534 1.99823542 0.76307924]


In [80]:
get_class_distribution(CAR_sent_class_df, True)

INS:  [0.54331738 6.27135527        inf]
INS, norm:  [ 0.  0. nan]
ISNS:  [0.73710066 2.50426741        inf]
ISNS, norm:  [ 0.  0. nan]
ENS:  [ 0.  0. nan]


/Users/filippakarrfelt/opt/anaconda3/envs/dp/lib/python3.7/site-packages/ipykernel_launcher.py:24: RuntimeWarning: divide by zero encountered in true_divide
/Users/filippakarrfelt/opt/anaconda3/envs/dp/lib/python3.7/site-packages/ipykernel_launcher.py:25: RuntimeWarning: divide by zero encountered in true_divide
/Users/filippakarrfelt/opt/anaconda3/envs/dp/lib/python3.7/site-packages/ipykernel_launcher.py:26: RuntimeWarning: invalid value encountered in true_divide
/Users/filippakarrfelt/opt/anaconda3/envs/dp/lib/python3.7/site-packages/ipykernel_launcher.py:30: RuntimeWarning: divide by zero encountered in true_divide
/Users/filippakarrfelt/opt/anaconda3/envs/dp/lib/python3.7/site-packages/ipykernel_launcher.py:34: RuntimeWarning: invalid value encountered in true_divide
/Users/filippakarrfelt/opt/anaconda3/envs/dp/lib/python3.7/site-packages/ipykernel_launcher.py:39: RuntimeWarning: divide by zero encountered in true_divide
/Users/filippakarrfelt/opt/anaconda3/envs/dp/lib/python3.7/s